In [1]:
import smtplib
from email.message import EmailMessage
from bs4 import BeautifulSoup
import requests

# 함수 정의

def create_soup(url):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"} 
    res = requests.get(url, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")
    return soup

def scrape_weather():
    weather_info = ""
    url = 'https://www.google.com/search?q=%EC%84%9C%EC%9A%B8%EB%82%A0%EC%94%A8&oq=%EC%84%9C%EC%9A%B8%EB%82%A0%EC%94%A8&gs_lcrp=EgZjaHJvbWUyDwgAEEUYORiDARixAxiABDIMCAEQABhDGIAEGIoFMgcIAhAAGIAEMgcIAxAAGIAEMgcIBBAAGIAEMgcIBRAAGIAEMgcIBhAAGIAEMgcIBxAAGIAEMgcICBAAGIAEMgcICRAAGIAEqAIAsAIA&sourceid=chrome&ie=UTF-8'
    soup = create_soup(url)

    cast = soup.find("span", attrs={"id": "wob_dc"}).get_text()
    cur_temp = soup.find("span", attrs={"class": "wob_t q8U8x"}).get_text()
    lowest = soup.find("div", class_='QrNVmd ZXCv8e')
    lowest_temp = lowest.find_all("span", class_='wob_t')[0].get_text()

    pp = soup.find("span", id='wob_pp').get_text()

    weather_info += "현재날씨: " + cast + "\n"
    weather_info += "현재온도: {} 오늘의 최저온도: {} 강수량: {}".format(cur_temp, lowest_temp, pp)

    return weather_info

def scrape_news():
    news_info = ""
    url = 'https://news.daum.net/'
    soup = create_soup(url)

    news_items = soup.find("ul", class_='list_newsissue').find_all("li", limit=5)
    for index, news_item in enumerate(news_items, start=1):
        strong_tag = news_item.find("strong", class_='tit_g')
        if strong_tag:
            title_text = strong_tag.get_text(strip=True)
        link = news_item.find("a")["href"]

        news_info += f"News {index}: {title_text}\n"
        news_info += f"Link: {link}\n\n"

    return news_info

import smtplib
from random import *
from account import *
from email.message import EmailMessage


# 메일 내용 생성
weather_info = scrape_weather()
news_info = scrape_news()
email_content = f"{weather_info}\n\n{news_info}"

# 이메일 보내기
msg = EmailMessage()
msg['Subject'] = '오늘의 날씨와 헤드라인 뉴스'
msg['From'] = EMAIL_ADDRESS
msg['To'] = 'dessian01@daum.net'  
msg.set_content(email_content)

# Gmail SMTP 서버 연결 및 메일 전송
with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
    smtp.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
    smtp.send_message(msg)
